In [23]:
import pandas as pd
import os
root = '../data/寶典/'
files = os.listdir(root)
for f in files:
    path = root+f
    xls = pd.ExcelFile(path)
    sheets = xls.sheet_names
    df = pd.read_excel(path,sheet_name=sheets[-1])[['CODIV','DIVNM','ITEMNM']].dropna(axis=0)
    print(f,df.shape)


寶典.v3.台塑網.20210901.xlsx (1099, 3)
寶典.v4.20211001.xlsx (1100, 3)
寶典.v5.20211006.xlsx (1103, 3)
寶典.v6.20211020.xlsx (1109, 3)
